In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas_profiling

from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

In [2]:
from google.colab import files
uploaded = files.upload()


Saving mort_table_1_2015_2019.csv to mort_table_1_2015_2019 (5).csv


DATASET LINK:https://data.gov.au/data/dataset/mort-books/resource/a5de4e7e-d062-4356-9d1b-39f44b1961dc

In [3]:
import io

In [41]:
df= pd.read_csv(io.BytesIO(uploaded['mort_table_1_2015_2019.csv']))

In [42]:
df.head()

,mort,category,geography,year,sex,deaths,population,"crude_rate_per_100,000","age_standardised_rate_per_100,000",rate_ratio,median_age,premature_deaths,premature_deaths_percent,"premature_deaths_asr_per_100,000",potential_years_of_life_lost,"pyll_rate_per_1,000",potentially_avoidable_deaths,pad_percent,"pad_asr_per_100,000"
0,REM1,Remoteness area,Major Cities of Australia,2015,Males,"51,943","8,401,989",618.2,621.1,0.95,79.0,"20,493",39.5,246.0,"350,184",44.0,"10,521",51.3,127.1
1,REM1,Remoteness area,Major Cities of Australia,2016,Males,"51,829","8,561,821",605.3,598.7,0.94,79.0,"20,146",38.9,235.9,"341,183",42.1,"10,315",51.2,121.7
2,REM1,Remoteness area,Major Cities of Australia,2017,Males,"52,968","8,737,440",606.2,593.3,0.95,79.0,"20,450",38.6,232.9,"343,185",41.5,"10,668",51.3,122.6
3,REM1,Remoteness area,Major Cities of Australia,2018,Males,"52,257","8,899,330",587.2,567.4,0.94,79.0,"20,381",39.0,226.3,"340,872",40.5,"10,263",49.7,115.1
4,REM1,Remoteness area,Major Cities of Australia,2019,Males,"55,965","9,062,381",617.6,587.9,0.94,79.0,"21,915",39.2,238.9,"373,552",43.7,"10,665",50.1,117.3


BASIC PREPROCESSING AND MANIPULATION


In [43]:
df.describe()

,year,"crude_rate_per_100,000","age_standardised_rate_per_100,000",rate_ratio,median_age,premature_deaths_percent,"premature_deaths_asr_per_100,000","pyll_rate_per_1,000",pad_percent,"pad_asr_per_100,000"
count,15795.000000,15542.000000,12768.000000,12768.000000,13760.000000,13760.000000,11400.000000,15542.000000,12800.000000,9644.000000
mean,2017.000000,725.076052,560.948457,1.048271,79.561773,37.045247,224.628175,48.222114,50.650164,119.056740
std,1.414258,257.086218,131.350428,0.192660,5.870464,13.534749,79.546792,27.175476,7.744214,48.889171
min,2015.000000,21.800000,226.900000,0.490000,44.000000,0.000000,63.800000,0.000000,7.700000,29.500000
25%,2016.000000,542.825000,466.500000,0.920000,77.000000,28.000000,166.200000,30.800000,46.400000,83.800000
50%,2017.000000,699.500000,546.600000,1.040000,80.500000,35.000000,213.350000,42.200000,50.200000,110.800000
75%,2018.000000,889.400000,638.900000,1.150000,83.000000,42.700000,269.200000,58.000000,54.500000,144.800000
max,2019.000000,2072.500000,1703.100000,2.670000,92.000000,100.000000,815.500000,396.300000,95.500000,483.100000


In [44]:
del df['mort']
del df['rate_ratio']

In [45]:
d={'category':'Areas','geography':'Regions','sex':'Gender','deaths':'Fatalities','population':'Inhabitants','crude_rate_per_100,000':'Fatalities_per_100000','age_standardised_rate_per_100,000':'Age_std_mortality','median_age':'MeanAge','premature_deaths':'PrematureFatalities','premature_deaths_percent':'PrematureFatalitiesPercent','premature_deaths_asr_per_100,000':'PrematureFatalityASR','potential_years_of_life_lost':'PotentialLifeLost','pyll_rate_per_1,000':'PLLper1000','potentially_avoidable_deaths':'PotentialAvoidableFatalities','pad_percent':'PAFpercent','pad_asr_per_100,000':'PAF_ASRper100000'}

In [46]:
df.rename(columns=d, inplace=True)

In [47]:
df

,Areas,Regions,year,Gender,Fatalities,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000,PotentialAvoidableFatalities,PAFpercent,PAF_ASRper100000
0,Remoteness area,Major Cities of Australia,2015,Males,"51,943","8,401,989",618.2,621.1,79.0,"20,493",39.5,246.0,"350,184",44.0,"10,521",51.3,127.1
1,Remoteness area,Major Cities of Australia,2016,Males,"51,829","8,561,821",605.3,598.7,79.0,"20,146",38.9,235.9,"341,183",42.1,"10,315",51.2,121.7
2,Remoteness area,Major Cities of Australia,2017,Males,"52,968","8,737,440",606.2,593.3,79.0,"20,450",38.6,232.9,"343,185",41.5,"10,668",51.3,122.6
3,Remoteness area,Major Cities of Australia,2018,Males,"52,257","8,899,330",587.2,567.4,79.0,"20,381",39.0,226.3,"340,872",40.5,"10,263",49.7,115.1
4,Remoteness area,Major Cities of Australia,2019,Males,"55,965","9,062,381",617.6,587.9,79.0,"21,915",39.2,238.9,"373,552",43.7,"10,665",50.1,117.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15790,Local Government Area (LGA),Australia (total),2015,Persons,"159,052","23,815,995",667.8,552.1,81.0,"53,881",33.9,214.0,"900,274",40.4,"27,458",51.0,110.8
15791,Local Government Area (LGA),Australia (total),2016,Persons,"158,504","24,190,907",655.2,536.0,81.0,"53,524",33.8,207.3,"883,157",39.1,"27,195",50.8,107.3
15792,Local Government Area (LGA),Australia (total),2017,Persons,"160,909","24,601,860",654.1,529.0,81.0,"54,079",33.6,203.7,"882,752",38.4,"27,847",50.8,107.1
15793,Local Government Area (LGA),Australia (total),2018,Persons,"158,493","24,982,688",634.4,508.2,81.0,"54,189",34.2,199.0,"871,807",37.4,"27,298",49.6,102.5


In [48]:
df.fillna(df.median())

,Areas,Regions,year,Gender,Fatalities,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000,PotentialAvoidableFatalities,PAFpercent,PAF_ASRper100000
0,Remoteness area,Major Cities of Australia,2015,Males,"51,943","8,401,989",618.2,621.1,79.0,"20,493",39.5,246.0,"350,184",44.0,"10,521",51.3,127.1
1,Remoteness area,Major Cities of Australia,2016,Males,"51,829","8,561,821",605.3,598.7,79.0,"20,146",38.9,235.9,"341,183",42.1,"10,315",51.2,121.7
2,Remoteness area,Major Cities of Australia,2017,Males,"52,968","8,737,440",606.2,593.3,79.0,"20,450",38.6,232.9,"343,185",41.5,"10,668",51.3,122.6
3,Remoteness area,Major Cities of Australia,2018,Males,"52,257","8,899,330",587.2,567.4,79.0,"20,381",39.0,226.3,"340,872",40.5,"10,263",49.7,115.1
4,Remoteness area,Major Cities of Australia,2019,Males,"55,965","9,062,381",617.6,587.9,79.0,"21,915",39.2,238.9,"373,552",43.7,"10,665",50.1,117.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15790,Local Government Area (LGA),Australia (total),2015,Persons,"159,052","23,815,995",667.8,552.1,81.0,"53,881",33.9,214.0,"900,274",40.4,"27,458",51.0,110.8
15791,Local Government Area (LGA),Australia (total),2016,Persons,"158,504","24,190,907",655.2,536.0,81.0,"53,524",33.8,207.3,"883,157",39.1,"27,195",50.8,107.3
15792,Local Government Area (LGA),Australia (total),2017,Persons,"160,909","24,601,860",654.1,529.0,81.0,"54,079",33.6,203.7,"882,752",38.4,"27,847",50.8,107.1
15793,Local Government Area (LGA),Australia (total),2018,Persons,"158,493","24,982,688",634.4,508.2,81.0,"54,189",34.2,199.0,"871,807",37.4,"27,298",49.6,102.5


In [49]:
df['Areas'].unique()

array(['Remoteness area', 'Socioeconomic group', 'State and territory',
       'Greater Capital City Statistical Areas (GCCSA)',
       'Primary Health Network (PHN)', 'Statistical Area Level 4 (SA4)',
       'Statistical Area Level 3 (SA3)', 'Local Government Area (LGA)'],
      dtype=object)

In [50]:
df['Regions'].unique()

array(['Major Cities of Australia', 'Inner Regional Australia',
       'Outer Regional Australia', ..., 'Unincorporated NT',
       'Unincorporated ACT', 'Unincorp. Other Territories'], dtype=object)

In [51]:
df['year'].unique()

array([2015, 2016, 2017, 2018, 2019])

In [52]:
from sklearn.preprocessing import LabelEncoder

In [53]:
labelencoder=LabelEncoder()
df['Area Code']=labelencoder.fit_transform(df['Areas'])
df['Year Code']=labelencoder.fit_transform(df['year'])
df['Gender Code']=labelencoder.fit_transform(df['Gender'])

In [54]:
df.insert(1,'Area Code',df.pop('Area Code'))
df.insert(3,'Year Code',df.pop('Year Code'))
df.insert(5,'Gender Code',df.pop('Gender Code'))


In [55]:
df

,Areas,Area Code,Regions,Year Code,year,Gender Code,Gender,Fatalities,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000,PotentialAvoidableFatalities,PAFpercent,PAF_ASRper100000
0,Remoteness area,3,Major Cities of Australia,0,2015,1,Males,"51,943","8,401,989",618.2,621.1,79.0,"20,493",39.5,246.0,"350,184",44.0,"10,521",51.3,127.1
1,Remoteness area,3,Major Cities of Australia,1,2016,1,Males,"51,829","8,561,821",605.3,598.7,79.0,"20,146",38.9,235.9,"341,183",42.1,"10,315",51.2,121.7
2,Remoteness area,3,Major Cities of Australia,2,2017,1,Males,"52,968","8,737,440",606.2,593.3,79.0,"20,450",38.6,232.9,"343,185",41.5,"10,668",51.3,122.6
3,Remoteness area,3,Major Cities of Australia,3,2018,1,Males,"52,257","8,899,330",587.2,567.4,79.0,"20,381",39.0,226.3,"340,872",40.5,"10,263",49.7,115.1
4,Remoteness area,3,Major Cities of Australia,4,2019,1,Males,"55,965","9,062,381",617.6,587.9,79.0,"21,915",39.2,238.9,"373,552",43.7,"10,665",50.1,117.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15790,Local Government Area (LGA),1,Australia (total),0,2015,2,Persons,"159,052","23,815,995",667.8,552.1,81.0,"53,881",33.9,214.0,"900,274",40.4,"27,458",51.0,110.8
15791,Local Government Area (LGA),1,Australia (total),1,2016,2,Persons,"158,504","24,190,907",655.2,536.0,81.0,"53,524",33.8,207.3,"883,157",39.1,"27,195",50.8,107.3
15792,Local Government Area (LGA),1,Australia (total),2,2017,2,Persons,"160,909","24,601,860",654.1,529.0,81.0,"54,079",33.6,203.7,"882,752",38.4,"27,847",50.8,107.1
15793,Local Government Area (LGA),1,Australia (total),3,2018,2,Persons,"158,493","24,982,688",634.4,508.2,81.0,"54,189",34.2,199.0,"871,807",37.4,"27,298",49.6,102.5


In [56]:
df.insert(8,'Percentage:Hypertension',np.random.uniform(10,12,df.shape[0]))
df.insert(9,'Percentage:Diabetes',np.random.uniform(4,6,df.shape[0]))




In [57]:
df

,Areas,Area Code,Regions,Year Code,year,Gender Code,Gender,Fatalities,Percentage:Hypertension,Percentage:Diabetes,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000,PotentialAvoidableFatalities,PAFpercent,PAF_ASRper100000
0,Remoteness area,3,Major Cities of Australia,0,2015,1,Males,"51,943",11.956442,4.075906,"8,401,989",618.2,621.1,79.0,"20,493",39.5,246.0,"350,184",44.0,"10,521",51.3,127.1
1,Remoteness area,3,Major Cities of Australia,1,2016,1,Males,"51,829",11.041608,5.078291,"8,561,821",605.3,598.7,79.0,"20,146",38.9,235.9,"341,183",42.1,"10,315",51.2,121.7
2,Remoteness area,3,Major Cities of Australia,2,2017,1,Males,"52,968",11.041908,5.763815,"8,737,440",606.2,593.3,79.0,"20,450",38.6,232.9,"343,185",41.5,"10,668",51.3,122.6
3,Remoteness area,3,Major Cities of Australia,3,2018,1,Males,"52,257",11.174558,4.877469,"8,899,330",587.2,567.4,79.0,"20,381",39.0,226.3,"340,872",40.5,"10,263",49.7,115.1
4,Remoteness area,3,Major Cities of Australia,4,2019,1,Males,"55,965",11.991986,4.171996,"9,062,381",617.6,587.9,79.0,"21,915",39.2,238.9,"373,552",43.7,"10,665",50.1,117.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15790,Local Government Area (LGA),1,Australia (total),0,2015,2,Persons,"159,052",11.239548,4.185055,"23,815,995",667.8,552.1,81.0,"53,881",33.9,214.0,"900,274",40.4,"27,458",51.0,110.8
15791,Local Government Area (LGA),1,Australia (total),1,2016,2,Persons,"158,504",10.415395,5.985348,"24,190,907",655.2,536.0,81.0,"53,524",33.8,207.3,"883,157",39.1,"27,195",50.8,107.3
15792,Local Government Area (LGA),1,Australia (total),2,2017,2,Persons,"160,909",10.962340,5.617088,"24,601,860",654.1,529.0,81.0,"54,079",33.6,203.7,"882,752",38.4,"27,847",50.8,107.1
15793,Local Government Area (LGA),1,Australia (total),3,2018,2,Persons,"158,493",10.595147,4.319579,"24,982,688",634.4,508.2,81.0,"54,189",34.2,199.0,"871,807",37.4,"27,298",49.6,102.5


In [58]:
df.drop("PotentialAvoidableFatalities",inplace=True,axis=1)


In [59]:
df.drop("Areas",inplace=True,axis=1)
df.drop("Regions",inplace=True,axis=1)
df.drop("year",inplace=True,axis=1)
df.drop("Gender",inplace=True,axis=1)

In [60]:
df.drop("PAF_ASRper100000",inplace=True,axis=1)

In [61]:
df["Fatalities"]=df["Fatalities"].str.replace(',', '').astype(float)
df["Inhabitants"]=df["Inhabitants"].str.replace(',', '').astype(float)
df["PrematureFatalities"]=df["PrematureFatalities"].str.replace(',', '').astype(float)
df["PotentialLifeLost"]=df["PotentialLifeLost"].str.replace(',', '').astype(float)


In [62]:
df.dropna()

,Area Code,Year Code,Gender Code,Fatalities,Percentage:Hypertension,Percentage:Diabetes,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000,PAFpercent
0,3,0,1,51943.0,11.956442,4.075906,8401989.0,618.2,621.1,79.0,20493.0,39.5,246.0,350184.0,44.0,51.3
1,3,1,1,51829.0,11.041608,5.078291,8561821.0,605.3,598.7,79.0,20146.0,38.9,235.9,341183.0,42.1,51.2
2,3,2,1,52968.0,11.041908,5.763815,8737440.0,606.2,593.3,79.0,20450.0,38.6,232.9,343185.0,41.5,51.3
3,3,3,1,52257.0,11.174558,4.877469,8899330.0,587.2,567.4,79.0,20381.0,39.0,226.3,340872.0,40.5,49.7
4,3,4,1,55965.0,11.991986,4.171996,9062381.0,617.6,587.9,79.0,21915.0,39.2,238.9,373552.0,43.7,50.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15790,1,0,2,159052.0,11.239548,4.185055,23815995.0,667.8,552.1,81.0,53881.0,33.9,214.0,900274.0,40.4,51.0
15791,1,1,2,158504.0,10.415395,5.985348,24190907.0,655.2,536.0,81.0,53524.0,33.8,207.3,883157.0,39.1,50.8
15792,1,2,2,160909.0,10.962340,5.617088,24601860.0,654.1,529.0,81.0,54079.0,33.6,203.7,882752.0,38.4,50.8
15793,1,3,2,158493.0,10.595147,4.319579,24982688.0,634.4,508.2,81.0,54189.0,34.2,199.0,871807.0,37.4,49.6


In [63]:
df['Percentage:Hypertension']=df['Percentage:Hypertension'].round(decimals=3)
df['Percentage:Hypertension']=df['Percentage:Diabetes'].round(decimals=3)




In [64]:
df =df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [65]:
df=df.round(0).astype(int)

Running Random Forest on original data(try to get <70% accuracy)
AFTER SOME BASIC MANIPULATION

In [66]:
X = df.drop("PAFpercent",1)   #Feature Matrix
y = df["PAFpercent"]          #Target Variable


In [67]:
np.isnan(X)


,Area Code,Year Code,Gender Code,Fatalities,Percentage:Hypertension,Percentage:Diabetes,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15790,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
15791,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
15792,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
15793,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [68]:
np.where(np.isnan(X))


(array([], dtype=int64), array([], dtype=int64))

In [69]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)



In [70]:
X_train=X_train.fillna(0)

In [71]:
y_train=y_train.fillna(0)

In [72]:
X_train

,Area Code,Year Code,Gender Code,Fatalities,Percentage:Hypertension,Percentage:Diabetes,Inhabitants,Fatalities_per_100000,Age_std_mortality,MeanAge,PrematureFatalities,PrematureFatalitiesPercent,PrematureFatalityASR,PotentialLifeLost,PLLper1000
5356,6,1,0,403,4,4,95394,422,438,82,139,34,152,2425,27
13526,1,1,2,901,4,4,80073,1125,566,83,244,27,236,3983,57
536,0,1,2,26430,5,5,4714387,561,488,82,8221,31,177,139292,32
4835,6,0,0,101,6,6,20297,498,440,81,37,37,175,425,22
11001,1,1,0,154,5,5,17102,900,424,85,41,27,166,575,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15574,1,4,2,520,5,5,57933,898,554,82,156,30,213,2270,43
5493,6,3,0,140,5,5,13950,1004,522,84,41,29,228,681,55
5722,6,2,0,61,5,5,8142,749,532,81,25,41,232,326,43
925,2,0,0,458,4,4,116778,392,667,64,312,68,314,7572,66


In [78]:
clf = RandomForestClassifier(n_estimators = 200) 

In [79]:
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 

from sklearn import metrics 
print()
 

print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.2675438596491228


In [77]:
df.to_csv('finaldata.csv')